# Rotating Drag Profile
This notebook is intended to connect with the xArm 7 and Bota Force/Torque Sensor. THen it will rotate the arm in set increments and record the drag force for a given duration. The forces and positions will be recorded as csv files in the data folder.

In [29]:
import os
from samplers import DataSource, PosSampler, FTSampler, Experiment
import threading
import time

from xarm.wrapper import XArmAPI
from util import rotate_forces
from scipy.signal import lfilter, lfilter_zi
import numpy as np

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from util import rotate_forces
from samplers import PosSampler, FTSampler, Experiment
from xarm.wrapper import XArmAPI
from scipy.signal import lfilter, lfilter_zi
from motion import fish_motion

Connect to the xArm and Bota force/torque sensor.

In [30]:
arm = XArmAPI('192.168.1.217')
arm.connect()

arm_sampler = PosSampler('arm', arm)
ft_sampler = FTSampler('FT', 'COM3')

ROBOT_IP: 192.168.1.217, VERSION: v2.3.0, PROTOCOL: V1, DETAIL: 7,7,XS1304,AC1303,v2.3.0, TYPE1300: [1, 1]
change protocol identifier to 3


Create and experiment that will take care of sampling from the sensors and saving the data.

In [31]:
experiment = Experiment([arm_sampler, ft_sampler])
experiment.connect_sensors()

BotaSerialSensorError: Could not open port

## Run an Experimental Trial
Change the values of the parameters to match the trial being conducted. The arm will move and the files will be named according to the values given.

In [52]:
arm.get_state()
arm.set_state(0)


0

### Move the Arm to the correct position for recording data

In [53]:
#_, angles = arm.get_servo_angle()
arm.set_servo_angle(angle=angles)

0

### Zero the sensor before turning on the flume

In [54]:
ft_sampler.zero()

Collecting...
CRC mismatch received
Frame synced
Collected for 10.001099109649658 seconds.


In [ ]:
# These values will control the experiment and be used to name the saveed data.
parameters = {
    'model': "Deep 60 Keel",
    'amplitude': 5,
    'frequency': None,
    'yaw_amplitude': 5,
    'yaw_frequency': None,
    'phase_shift': 0,
    'flume-speed': 0,
   
    
}
# sleep_duration should be long enough to allow the system to reach steady state before any data is recorded and duration is ho long you would like to record data for. Both times are in seconds.
# sleep_duration = 10
# duration = 4
amplitude=parameters['amplitude']
yaw_amplitude=parameters['yaw_amplitude']
phase_shift=parameters['phase_shift']

arm.set_state(0)
for frequency in [0.5,1,1.5,2,2.5,3]:
    duration=int (round(10/frequency))*2
    yaw_frequency=frequency
    motion_thread=threading.Thread(target=fish_motion,args=(arm,amplitude,frequency,yaw_amplitude,yaw_frequency,phase_shift,duration))
    motion_thread.start()
    # The arm will move to the given location
    parameters['frequency'] = frequency
    parameters['yaw_frequency'] = yaw_frequency
    

    # Sleep while the system reaches steady state
    time.sleep(duration//2)

    # Record Data
    print('Starting recording...')
    experiment.new_trial(parameters, duration=duration//2)
    print('Finished recording...')
    time.sleep(3)
    

    
    # window_size = 450
    # b = 1 / window_size * np.ones(window_size)
    # a = 1
    # zi = lfilter_zi(b, a)
    
    # force_file = position_file = experiment.get_dir_from_parameters()
    # force_file += '_FT/FT.csv'
    # position_file += '_arm/arm.csv'
    # df = rotate_forces(force_file, position_file)
    # x, _ = lfilter(b, a, df['Fx'], zi=zi*df['Fx'].mean())
    # y, _ = lfilter(b, a, df['Fy'], zi=zi*df['Fy'].mean())
    # z, _ = lfilter(b, a, df['Fz'], zi=zi*df['Fz'].mean())
    # df['Filtered x'] = x
    # df['Filtered y'] = y
    # df['Filtered z'] = z
    # os.mkdir(experiment.get_dir_from_parameters()+'_rotated')
    # df.to_csv(experiment.get_dir_from_parameters()+'_rotated/force.csv', index=False)
    motion_thread.join()
    


Starting recording...
Finished recording...
Starting recording...
Finished recording...
Starting recording...
Finished recording...
Starting recording...
Finished recording...
Starting recording...
Finished recording...
Starting recording...
Finished recording...


[SDK][ERROR][2024-08-12 12:54:46][base.py:293] - - [main-socket] recv error: [WinError 10054] An existing connection was forcibly closed by the remote host
[SDK][ERROR][2024-08-12 12:54:46][base.py:293] - - [main-socket] recv error: [WinError 10054] An existing connection was forcibly closed by the remote host
[SDK][ERROR][2024-08-12 12:54:46][base.py:293] - - [main-socket] recv error: [WinError 10054] An existing connection was forcibly closed by the remote host


Exception in thread Thread-11 (_processdata_thread):
Traceback (most recent call last):
  File "c:\Users\donatell\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1052, in _bootstrap_inner
    self.run()
  File "c:\Users\donatell\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 989, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\donatell\OneDrive - Chapman University\Teaching\CPSC230_Spring2024\Projects\Project Part 3\FHL-Projects\samplers.py", line 266, in _processdata_thread
    crc16_ccitt_frame = self._ser.read(2)
                        ^^^^^^^^^^^^^^^^^
  File "c:\Users\donatell\AppData\Local\Programs\Python\Python312\Lib\site-packages\serial\serialwin32.py", line 275, in read
    raise SerialException("ClearCommError failed ({!r})".format(ctypes.WinError()))
serial.serialutil.SerialException: ClearCommError failed (PermissionError(13, 'Access is denied.', None, 5))


[SDK][ERROR][2024-08-12 12:54:47][base.py:176] - - [report-socket] socket read failed, len=0
[SDK][ERROR][2024-08-12 12:54:47][base.py:176] - - [report-socket] socket read failed, len=0
[SDK][ERROR][2024-08-12 12:54:47][base.py:176] - - [report-socket] socket read failed, len=0
[SDK][ERROR][2024-08-12 12:54:48][base.py:1213] - - client timeout over 90s, disconnect
[SDK][ERROR][2024-08-12 12:54:48][base.py:1213] - - client timeout over 90s, disconnect
[SDK][ERROR][2024-08-12 12:54:48][base.py:1213] - - client timeout over 90s, disconnect
[SDK][ERROR][2024-08-12 12:54:48][decorator.py:39] - - xArm is not connected
[SDK][ERROR][2024-08-12 12:54:48][decorator.py:39] - - xArm is not connected


Exception in thread Thread-66 (_processdata_thread):
Traceback (most recent call last):
  File "c:\Users\donatell\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1052, in _bootstrap_inner
    self.run()
  File "c:\Users\donatell\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 989, in run
Exception in thread Thread-10 (_processdata_thread):
Traceback (most recent call last):
  File "c:\Users\donatell\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1052, in _bootstrap_inner
    self._target(*self._args, **self._kwargs)
  File "c:\Users\donatell\OneDrive - Chapman University\Teaching\CPSC230_Spring2024\Projects\Project Part 3\FHL-Projects\samplers.py", line 63, in _processdata_thread
    self.run()
  File "c:\Users\donatell\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 989, in run
    state, (x, y, z, theta_x, theta_y, theta_z) = self.arm.get_position()
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: too many va